In [1]:
!pip install fastapi uvicorn streamlit pyngrok pandas requests prophet tsfresh scikit-learn matplotlib    --quiet

In [2]:
from pyngrok import ngrok
ngrok.set_auth_token("2vgULFP5rjcVsMkwGpjJUGvyct2_9nT4An12iTiYqTZStQPf")

In [3]:
%%writefile backend.py

from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
import pandas as pd
import numpy as np

from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

app = FastAPI(title="FitPulse – Health Anomaly Backend")

# ================= GLOBAL STATE =================
CLEAN_DF = None
FEATURE_DF = None
ALERTS_DF = None

# ==================================================
# MODULE 1 – PREPROCESSING
# ==================================================
@app.post("/preprocess")
def preprocess(file: UploadFile = File(...)):
    global CLEAN_DF

    df = pd.read_csv(file.file)

    REQUIRED_COLUMNS = [
        "user_id",
        "date",
        "TotalSteps",
        "avg_heart_rate",
        "total_sleep_minutes"
    ]

    missing = [c for c in REQUIRED_COLUMNS if c not in df.columns]
    if missing:
        raise HTTPException(
            status_code=400,
            detail=f"Missing required columns: {missing}"
        )

    # Select & rename only needed columns
    df = df[REQUIRED_COLUMNS].rename(columns={
        "TotalSteps": "steps",
        "avg_heart_rate": "heart_rate",
        "total_sleep_minutes": "sleep_minutes"
    })

    REQUIRED_COLUMNS = [
        "user_id",
        "date",
        "TotalSteps",
        "avg_heart_rate",
        "total_sleep_minutes"
    ]   # Check required columns


    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"])

    df["steps"] = pd.to_numeric(df["steps"], errors="coerce").fillna(0)
    df["heart_rate"] = pd.to_numeric(df["heart_rate"], errors="coerce")
    df["sleep_minutes"] = pd.to_numeric(df["sleep_minutes"], errors="coerce")

    df["heart_rate"].fillna(df["heart_rate"].median(), inplace=True)
    df["sleep_minutes"].fillna(df["sleep_minutes"].median(), inplace=True)

    # Convert sleep to hours
    df["sleep_hours"] = df["sleep_minutes"] / 60

    # Daily aggregation
    df = (
        df.set_index("date")
        .groupby("user_id")[["heart_rate", "steps", "sleep_hours"]]
        .resample("D")
        .mean()
        .reset_index()
    )

    CLEAN_DF = df
    df.to_csv("clean_data.csv", index=False)

    return {"rows": len(df)}

# ==================================================
# MODULE 2 – FEATURE EXTRACTION + MODELING
# ==================================================
@app.post("/module2")
def module2():
    global CLEAN_DF, FEATURE_DF

    if CLEAN_DF is None:
        raise HTTPException(status_code=400, detail="Run preprocessing first")

    df = CLEAN_DF.sort_values(["user_id", "date"])

    df["hr_7d_mean"] = df.groupby("user_id")["heart_rate"].transform(
        lambda x: x.rolling(7, min_periods=1).mean()
    )

    df["steps_7d_mean"] = df.groupby("user_id")["steps"].transform(
        lambda x: x.rolling(7, min_periods=1).mean()
    )

    FEATURE_DF = df.copy()

    # Prophet baseline (heart rate)
    plot = []
    p_df = (
        df.groupby("date")["heart_rate"]
        .mean()
        .reset_index()
        .rename(columns={"date": "ds", "heart_rate": "y"})
    )

    if len(p_df) > 10:
        m = Prophet()
        m.fit(p_df)
        f = m.predict(p_df)
        p_df["yhat"] = f["yhat"]
        plot = p_df.tail(30).to_dict("records")

    # DBSCAN clustering
    X = FEATURE_DF[["heart_rate", "steps", "sleep_hours", "hr_7d_mean"]].fillna(0)
    X = StandardScaler().fit_transform(X)

    db = DBSCAN(eps=1.2, min_samples=5)
    FEATURE_DF["cluster"] = db.fit_predict(X)
    FEATURE_DF["cluster_anomaly"] = FEATURE_DF["cluster"] == -1

    FEATURE_DF.to_csv("feature_data.csv", index=False)

    return {"heart_rate_plot": plot}

# ==================================================
# MODULE 3 – ANOMALY DETECTION
# ==================================================
@app.post("/module3")
def module3():
    global FEATURE_DF, ALERTS_DF

    if FEATURE_DF is None:
        raise HTTPException(status_code=400, detail="Run Module 2 first")

    rows = []

    for _, r in FEATURE_DF.iterrows():
        if r["heart_rate"] > 120:
            rows.append((r["user_id"], r["date"], "High Heart Rate"))
        if r["heart_rate"] < 40:
            rows.append((r["user_id"], r["date"], "Low Heart Rate"))
        if r["sleep_hours"] < 4 or r["sleep_hours"] > 12:
            rows.append((r["user_id"], r["date"], "Abnormal Sleep"))
        if r["cluster_anomaly"]:
            rows.append((r["user_id"], r["date"], "Behavioral Outlier"))

    df = pd.DataFrame(rows, columns=["user_id", "date", "metric"])

    alerts = (
        df.groupby(["user_id", "metric"])
        .agg(count=("date", "count"))
        .reset_index()
    )

    alerts["severity"] = alerts["count"].apply(
        lambda x: "High" if x >= 5 else "Medium" if x >= 3 else "Low"
    )

    ALERTS_DF = alerts
    alerts.to_csv("module3_alerts.csv", index=False)

    return {"alerts": alerts.to_dict("records")}

# ==================================================
# MODULE 4 – INSIGHTS
# ==================================================
@app.post("/module4")
def module4():
    if ALERTS_DF is None or ALERTS_DF.empty:
        raise HTTPException(status_code=400, detail="Run Module 3 first")

    insights = [
        {
            "user_id": r["user_id"],
            "severity": r["severity"],
            "insight": f"User {r['user_id']} shows {r['severity']} risk due to {r['metric']}"
        }
        for _, r in ALERTS_DF.iterrows()
    ]

    return {"insights": insights}


Overwriting backend.py


In [4]:
# Kill any process using port 8000 before starting FastAPI
!fuser -k 8000/tcp

from pyngrok import ngrok
import uvicorn
import threading
import time

threading.Thread(
    target=uvicorn.run,
    kwargs={
        "app": "backend:app",
        "host": "0.0.0.0",
        "port": 8000,
        "log_level": "info"
    },
    daemon=True
).start()
time.sleep(3) # Give FastAPI time to start
public_url = ngrok.connect(8000)
print("FastAPI URL:", public_url)

INFO:     Started server process [39758]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


FastAPI URL: NgrokTunnel: "https://593f0c86f9f6.ngrok-free.app" -> "http://localhost:8000"


In [14]:
%%writefile app.py
import streamlit as st
import requests
import pandas as pd
import plotly.express as px

# ================= BACKEND URL =================
BACKEND_URL = "http://localhost:8000"

# ================= PAGE CONFIG =================
st.set_page_config(
    page_title="FitPulse – Health Anomaly Detection",
    layout="wide"
)

# ================= UI STYLING =================
st.markdown("""
<style>

/* ---------- GLOBAL ---------- */
.stApp {
    background-color: #f4f6fb;
    color: #1f2933;
}
html, body, [class*="css"] {
    color: #1f2933 !important;
}

/* ---------- HEADINGS ---------- */
h1 {
    color: #1e3a8a;
    font-weight: 800;
}
h2, h3 {
    color: #1e40af;
}

/* ---------- SIDEBAR ---------- */
section[data-testid="stSidebar"] {
    background-color: #e0e7ff;
    border-right: 2px solid #c7d2fe;
}
section[data-testid="stSidebar"] * {
    color: #1f2933 !important;
}

/* ---------- FILE UPLOADER ---------- */
div[data-testid="stFileUploader"] {
    background-color: #ffffff;
    border: 2px dashed #6366f1;
    border-radius: 14px;
    padding: 18px;
}

/* ---------- BUTTONS ---------- */
button {
    background-color: #6366f1 !important;
    color: white !important;
    border-radius: 10px;
    border: none;
    font-weight: 600;
}
button:hover {
    background-color: #4f46e5 !important;
}

/* ---------- METRIC CARDS ---------- */
div[data-testid="metric-container"] {
    background-color: #ffffff;
    padding: 18px;
    border-radius: 16px;
    box-shadow: 0 6px 18px rgba(0,0,0,0.08);
    border-left: 6px solid #6366f1;
}

/* ---------- TABS FIX (IMPORTANT) ---------- */
div[data-baseweb="tab-list"] {
    gap: 10px;
    padding: 10px 0 20px 0;
    justify-content: flex-start;
}

button[data-baseweb="tab"] {
    background-color: #e0e7ff !important;
    color: #1e3a8a !important;
    border-radius: 10px !important;
    padding: 10px 18px !important;
    font-weight: 600;
    border: none !important;
}

/* Active tab */
button[data-baseweb="tab"][aria-selected="true"] {
    background-color: #6366f1 !important;
    color: white !important;
}

/* Remove weird underline */
button[data-baseweb="tab"] > div {
    border: none !important;
}

/* ---------- ALERT BOXES ---------- */
div[data-testid="stAlert"] {
    border-radius: 12px;
    font-size: 15px;
}

/* ---------- TABLE ---------- */
thead tr th {
    background-color: #c7d2fe !important;
    color: #1f2933 !important;
}
tbody tr td {
    background-color: #ffffff !important;
    color: #1f2933 !important;
}

</style>
""", unsafe_allow_html=True)


# ================= HEADER =================
st.title("FitPulse Health Monitoring Dashboard")
st.caption("AI-powered detection of unusual health patterns from fitness device data")
st.divider()

# ================= SIDEBAR =================
st.sidebar.header("Upload Fitness Data")

uploaded_file = st.sidebar.file_uploader(
    "Upload CSV file",
    type=["csv"]
)

if uploaded_file:
    with st.spinner("Running preprocessing..."):
        res = requests.post(
            f"{BACKEND_URL}/preprocess",
            files={"file": uploaded_file}
        )

    if res.status_code == 200:
        st.sidebar.success("Data processed successfully")
        st.sidebar.metric("Records Loaded", res.json()["rows"])
    else:
        st.sidebar.error(res.text)

# ================= LOAD CLEAN DATA =================
@st.cache_data
def load_clean():
    return pd.read_csv("clean_data.csv", parse_dates=["date"])

try:
    df = load_clean()
except:
    st.info("Upload data to activate dashboard")
    st.stop()

# ================= FILTERS =================
st.subheader("Filters")
c1, c2, c3 = st.columns(3)

with c1:
    users = ["All"] + sorted(df["user_id"].unique())
    selected_user = st.selectbox("User", users)

with c2:
    start_date, end_date = st.date_input(
        "Date Range",
        value=[df["date"].min(), df["date"].max()]
    )

with c3:
    metric = st.selectbox(
        "Health Metric",
        ["Heart Rate", "Sleep", "Steps"]
    )

if selected_user != "All":
    df = df[df["user_id"] == selected_user]

df = df[
    (df["date"] >= pd.to_datetime(start_date)) &
    (df["date"] <= pd.to_datetime(end_date))
]

st.divider()

# ================= TABS =================
tab1, tab2, tab3, tab4 = st.tabs([
    "Overview",
    "Trends & Patterns",
    "Anomalies",
    "Insights & Reports"
])

# ================= TAB 1 =================
with tab1:
    st.subheader("Data Overview")
    c1, c2, c3 = st.columns(3)
    c1.metric("Users", df["user_id"].nunique())
    c2.metric("Days Tracked", df["date"].nunique())

    if metric == "Heart Rate":
        y_col = "heart_rate"
        c3.metric("Avg HR", round(df[y_col].mean(), 1))
    elif metric == "Sleep":
        y_col = "sleep_hours"
        c3.metric("Avg Sleep (hrs)", round(df[y_col].mean(), 2))
    else:
        y_col = "steps"
        c3.metric("Avg Steps", int(df[y_col].mean()))

    graph_type = st.selectbox(
        "Select Graph Type",
        ["Line (Recommended)", "Area", "Bar"]
    )

    if graph_type == "Line (Recommended)":
        fig = px.line(df, x="date", y=y_col, color="user_id")
    elif graph_type == "Area":
        fig = px.area(df, x="date", y=y_col, color="user_id")
    else:
        fig = px.bar(df, x="date", y=y_col, color="user_id")

    fig.update_layout(template="plotly_white")
    st.plotly_chart(fig, use_container_width=True)

# ================= TAB 2 =================
with tab2:
    st.subheader("Health Pattern Analysis")

    if st.button("Run Analysis"):
        with st.spinner("Running analysis..."):
            res = requests.post(f"{BACKEND_URL}/module2")

        if res.status_code == 200:
            hr_plot = res.json().get("heart_rate_plot", [])
            if hr_plot:
                pdf = pd.DataFrame(hr_plot)
                fig = px.line(pdf, x="ds", y=["y", "yhat"])
                st.plotly_chart(fig, use_container_width=True)
        else:
            st.error(res.text)

# ================= TAB 3 =================
with tab3:
    st.subheader("Detected Health Anomalies")

    if st.button("Detect Anomalies"):
        with st.spinner("Detecting anomalies..."):
            res = requests.post(f"{BACKEND_URL}/module3")

        if res.status_code == 200:
            alerts = pd.DataFrame(res.json()["alerts"])
            st.dataframe(alerts, use_container_width=True)

            fig = px.bar(
                alerts,
                x="metric",
                y="count",
                color="severity"
            )
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.error(res.text)

# ================= TAB 4 =================
with tab4:
    st.subheader("Health Insights")

    if st.button("Generate Insights"):
        with st.spinner("Generating insights..."):
            res = requests.post(f"{BACKEND_URL}/module4")

        if res.status_code == 200:
            for i in res.json()["insights"]:
                st.warning(f"{i['insight']}")
        else:
            st.error(res.text)

    st.divider()
    st.subheader("Downloads")

    try:
        st.download_button(
            "Download Clean Data",
            data=open("clean_data.csv", "rb"),
            file_name="clean_data.csv"
        )

        st.download_button(
            "Download Alerts Report",
            data=open("module3_alerts.csv", "rb"),
            file_name="health_alerts.csv"
        )
    except:
        st.info("Run analysis and anomaly detection to enable downloads.")


Overwriting app.py


In [15]:
# Kill any process using port 8501 before starting Streamlit
!fuser -k 8501/tcp
!streamlit run app.py &>/content/logs.txt &

8501/tcp:            40757


In [16]:
from pyngrok import ngrok
import time
ngrok.kill()
time.sleep(2) # Add a small delay to ensure previous ngrok session is terminated
streamlit_url = ngrok.connect(8501)
print("Streamlit public URL:", streamlit_url)

Streamlit public URL: NgrokTunnel: "https://03522ed94d71.ngrok-free.app" -> "http://localhost:8501"


In [17]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0



2026-01-13 14:30:05.429 Port 8501 is already in use
